In [195]:
pip install pyTelegramBotAPI

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [196]:
import telebot
import webbrowser
from telebot import types
import sqlite3

In [197]:
bot = telebot.TeleBot('5943866957:AAFThnNXwb1tFfbhx6TCQ5to7zZY-P53BjA')
name = None

In [198]:
@bot.message_handler(commands=['start'])
def start(message):
    conn = sqlite3.connect('Base.sql')
    cur = conn.cursor()

    cur.execute('CREATE TABLE IF NOT EXISTS users (id int auto_increment primary key, name varchar(50), pass varchar(50))')
    conn.commit()
    cur.close()
    conn.close()
    
    def user_name(message):
        mess = f'Привет, {message.from_user.first_name}, тебе нужно пройти небольшую регистрацию, начнем с имени'
        global name
        name = message.text.strip()
        bot.send_message(message.chat.id, 'Теперь пароль')
        bot.register_next_step_handler(message, user_pass)
    bot.send_message(message.chat.id, mess, parse_mode='html')
    bot.register_next_step_handler(message, user_name)

    

    def user_pass(message):
        password = message.text.strip()

        conn = sqlite3.connect('Base.sql')
        cur = conn.cursor()

        cur.execute("INSERT INTO users (name, pass) VALUES ('%s', '%s')" % (name, password)) 
        conn.commit()
        cur.close()
        conn.close()
        markup = types.InlineKeyboardMarkup()
        markup.add(types.InlineKeyboardButton('Список пользователей', callback_data='users'))
        bot.send_message(message.chat.id, "Вы заргестрированы!", reply_markup=markup)
    


@bot.message_handler(commands=['website'])
def get_user_photo(message):
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton("Посетить веб сайт", url="https://translate.yandex.ru/?source_lang=en&target_lang=ru&text=%27list%27%20object%20is%20not%20callable"))
    bot.send_message(message.chat.id, 'Переходи и переводи!', reply_markup=markup)
    
@bot.message_handler(commands=['site'])
def site(message):
    markup = types.InlineKeyboardMarkup()
    webbrowser.open('https://translate.yandex.ru/?source_lang=en&target_lang=ru&text=%27list%27%20object%20is%20not%20callable')

@bot.message_handler(commands=['help'])
def website(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)  #Кнопки статичные
    website = types.KeyboardButton('Веб сайт')
    start = types.KeyboardButton('Start')
    markup.add(website, start)
    bot.send_message(message.chat.id, reply_markup=markup)

@bot.message_handler(content_types=['photo'])
def get_user_photo(message):
    markup = types.InlineKeyboardMarkup()  #Кнопки для текста/ответа
    btn1 = types.InlineKeyboardButton('Нажми меня 🤡', callback_data='joke')
    markup.row(btn1)
    btn2 = types.InlineKeyboardButton('Удалить фото ❌', callback_data='delete')
    btn3 = types.InlineKeyboardButton('Изменить текст ✏️', callback_data='edit')
    markup.row(btn2, btn3)
    bot.send_message(message.chat.id, "Вау, крутое фото!", reply_markup=markup)

@bot.callback_query_handler(func=lambda callback: True)
def callback_message(callback):
    if callback.data == 'delete':
        bot.delete_message(callback.message.chat.id, callback.message.message_id - 1)
    elif callback.data == 'edit':
        bot.edit_message_text('Edit text', callback.message.chat.id, callback.message.message_id)
    
@bot.message_handler()
def get_user_text(message):
    if message.text.lower() == "привет":
        bot.send_message(message.chat.id, "Ну привет", parse_mode='html')
    elif message.text.lower() == 'id':
        bot.send_message(message.chat.id, f"Твой ID: {message.from_user.id}", parse_mode='html')
    elif message.text.lower() == 'inf':
        bot.send_message(message.chat.id, f"Информация о сообщении: {message}", parse_mode='html')
    elif message.text.lower() == "photo":
        photo = open('123.png', 'rb')
        bot.send_photo(message.chat.id, photo)
    else:
        bot.send_message(message.chat.id, "Что ты там пишешь?", parse_mode='html')




bot.polling(non_stop=True)